# Title of the analysis:

# *Analytical description of ABC DEF GHI.*

# Description of the dataset's columns:

9 of a total of 11 relevant datasets can be found on this link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/

2 of a total of 11 relevant datasets can be found on this link: https://www.kaggle.com/datasets/olistbr/marketing-funnel-olist/

# Index of the analysis:

<ins> 1. Data Preprocessing for Exploratory Data Analysis </ins>

- 1.a) Top-level understanding of the dataset

- 1.b) Treatment of missing values

- 1.c) Elimination of duplicate rows

- 1.d) Transformation of categorical variables into numerical ones

- 1.e) Feature engineering

<ins> 2. Exploratory Data Analysis </ins>

- 2.a) Histogram display for ABC DEF GHI

- 2.b) Boxplot display for ABC DEF GHI

- 2.c) Relation between selected independent and dependent variables of the dataset ABC DEF GHI

- 2.d) Pivot table indicating the top 20 orders with the greatest ABC DEF GHI

<ins> 3. Data Preprocessing for Machine Learning </ins>

- 3.a) Outlier elimination

- 3.b) Distribution fitting via standardization and dimensionality reduction via factor analysis

- 3.c) Canonical correlation analysis or PCA, Ill have to see ABC DEF GHI

<ins> 4. Multivariate Analysis Of Variance (MANOVA) </ins>

<ins> 5. Machine Learning Model predicting ABC DEF GHI </ins>

# 1. Data Preprocessing for Exploratory Data Analysis

## <ins>1.a) Top-level understanding and treatment of the dataset</ins>

After transforming each analyzable dataset into a dataframe (df) - which is a necessary step to enable that very analysis - we’ll cluster the rows in the Geolocation df’s columns `geolocation_lat`, `geolocation_lng`, `geolocation_city` and `geolocation_state` to effectively eliminate unnecessarily granular information that’d complexify our analysis with no great benefits. Then we’ll merge all of those dataframes into one, maintaining all of the customers’ orders regardless of them having information on the data datasets that pertains to them.   

In [1]:
import pandas as pd

# Identifying the file paths of all relevant Olis' databases:

customers_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_customers_dataset.csv'
geolocation_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_geolocation_dataset.csv'
order_items_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_order_items_dataset.csv'
order_payments_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_order_payments_dataset.csv'
order_reviews_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_order_reviews_dataset.csv'
orders_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_orders_dataset.csv'
products_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_products_dataset.csv'
sellers_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/olist_sellers_dataset.csv'
product_category_name_translation_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Main databases/product_category_name_translation.csv'
closed_deals_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Marketing databases/olist_closed_deals_dataset.csv'
marketing_qualified_leads_dataset_file_path = '/Users/goncalolazaro/Downloads/Olist databases/Marketing databases/olist_marketing_qualified_leads_dataset.csv'

# Giving indication to display all columns of Olis' databases - instead of a selected few - when database display's asked for:

pd.set_option('display.max_columns', None)

# Creating dataframes for each and every one of Olis' databases:

customers_df = pd.read_csv(customers_dataset_file_path, engine='python', on_bad_lines='skip')
geolocation_df = pd.read_csv(geolocation_dataset_file_path)
order_items_df = pd.read_csv(order_items_dataset_file_path)
order_payments_df = pd.read_csv(order_payments_dataset_file_path)
order_reviews_df = pd.read_csv(order_reviews_dataset_file_path)
orders_df = pd.read_csv(orders_dataset_file_path)
products_df = pd.read_csv(products_dataset_file_path)
sellers_df = pd.read_csv(sellers_dataset_file_path)
product_category_name_translation_df = pd.read_csv(product_category_name_translation_dataset_file_path)
closed_deals_df = pd.read_csv(closed_deals_dataset_file_path)
marketing_qualified_leads_df = pd.read_csv(marketing_qualified_leads_dataset_file_path)

<ins> Note about the reason for the customers dataset to have had lines skipped: </ins>

The customer dataset couldn't have been read by the currently-used IDE Jupyter Lab without either skipping or altering the IDE's interpretation of precisely one erroneously-filled row inside it, that row referring to exactly one customer. Considering that only the very insignificant amount of one row is erroneous in a universe of 4053 rows (virtually 0% of total customers) justified my decision of skipping the problematic line instead of applying the larger and inconsequential effort of altering it.

In [2]:
# Grouping rows on the Geolocation dataframe that essentially present the same information with only insignificant differences in longitude and latitude

geolocation_df = geolocation_df[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']]

zip_grouped_geolocation_df = geolocation_df.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'geolocation_city': 'first',   
    'geolocation_state': 'first'   
}).reset_index()

print(zip_grouped_geolocation_df.head())

   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1001       -23.550190       -46.634024   
1                         1002       -23.548146       -46.634979   
2                         1003       -23.548994       -46.635731   
3                         1004       -23.549799       -46.634757   
4                         1005       -23.549456       -46.636733   

  geolocation_city geolocation_state  
0        sao paulo                SP  
1        sao paulo                SP  
2        sao paulo                SP  
3        sao paulo                SP  
4        sao paulo                SP  


<ins> Note about the reason for all rows on the Geolocation dataframe that pertained to one single zip code having been grouped into one: </ins>

On the original geolocation_df, each zip code would often appear on several rows, each indicating a different specific locality (latitude & longitude) inside that very zip code. Such a wide and specific range of information would unecessarily add complexity to the final dataframe I'm building in the next step of this analysis, as the differences in latitude and longitude inside each code are insignificant considering the scope of my analysis, hence me calculating each zip code's mean of (latitude,longitude) and using that datapoint for forthcoming analyses.  

In [3]:
import dask.dataframe as dd

# Merging all of Olis' databases into one:

orders_customers_df = dd.merge(orders_df, customers_df, on='customer_id', how='left')
orders_customers_items_df = dd.merge(orders_customers_df, order_items_df, on='order_id', how='left')
orders_customers_items_payments_df = dd.merge(orders_customers_items_df, order_payments_df, on='order_id', how='left')
orders_customers_items_payments_reviews_df = dd.merge(orders_customers_items_payments_df, order_reviews_df, on='order_id', how='left')
orders_customers_items_payments_reviews_products_df = dd.merge(orders_customers_items_payments_reviews_df, products_df, on='product_id', how='left')
orders_customers_items_payments_reviews_products_sellers_df = dd.merge(orders_customers_items_payments_reviews_products_df, sellers_df, on='seller_id', how='left')
orders_customers_items_payments_reviews_products_sellers_geolocation_df = dd.merge(orders_customers_items_payments_reviews_products_sellers_df, zip_grouped_geolocation_df, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
orders_full_df = dd.merge(orders_customers_items_payments_reviews_products_sellers_geolocation_df, product_category_name_translation_df, on='product_category_name', how='left')
marketing_full_df = dd.merge(marketing_qualified_leads_df, closed_deals_df, on='mql_id', how='left')
df = dd.merge(orders_full_df, marketing_full_df, on='seller_id', how='left')

In [4]:
df.head(20)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,product_category_name_english,mql_id,first_contact_date,landing_page_id,origin,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,<NA>,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,<NA>,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,<NA>,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,31570.0,-19.807681,-43.980427,belo horizonte,MG,perfumery,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265.0,vianopolis,GO,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,1.0,credi

## <ins>1.b) Treatment of missing values</ins>

Because all the commonly-used chart-creating libraries in Python already perfectly handle NaN, null or straight missing values, there is no reason to transform currently missing cells into anything else unless a value for those cells could be inferred from existing data that’d enrich our analysis versus leaving the missing cell as is. In cases for which numeric or string values are inferred to replace missing ones, we’ll add new columns to the dataset flagging the original null values in order to ensure transparency in how those gaps were addressed.

In [5]:
# Understanding the quantity of null values inside each of the final dataframe's columns:

null_counts = df.isnull().sum().compute()

for column, count in null_counts.items():
    print(f'{column}: {count}')

order_id: 0
customer_id: 0
order_status: 0
order_purchase_timestamp: 0
order_approved_at: 1159611
order_delivered_carrier_date: 5956710
order_delivered_customer_date: 5965202
order_estimated_delivery_date: 0
customer_unique_id: 5825567
customer_zip_code_prefix: 5825567
customer_city: 5825567
customer_state: 5825567
order_item_id: 5962614
product_id: 5962614
seller_id: 5962614
shipping_limit_date: 5962614
price: 5962614
freight_value: 5962614
payment_sequential: 3
payment_type: 3
payment_installments: 3
payment_value: 3
review_id: 136980
review_score: 136980
review_comment_title: 5372706
review_comment_message: 2001288
review_creation_date: 136980
review_answer_timestamp: 136980
product_category_name: 5964324
product_name_lenght: 5962634
product_description_lenght: 5964324
product_photos_qty: 5962634
product_weight_g: 5962634
product_length_cm: 5962634
product_height_cm: 5962634
product_width_cm: 5962634
seller_zip_code_prefix: 5962614
seller_city: 5962614
seller_state: 5962614
geolocat

## How I'm treating the missing data

- For column "order approved at", I first created a new column (`order_approved_at_null_flag`) to flag the rows where the value was originally missing (1 if NaN, 0 otherwise). Next, I calculated the dataset's average time difference between when the order was placed (`order_purchase_timestamp`) and when it was approved (`order_approved_at`) (with each order being a single data point in said calculation). Finally, I used that metric to fill in the missing values in the `order_approved_at` column by adding it to the corresponding `order_purchase_timestamp`.

- For columns "order_delivered_carrier_date" and "order_delivered_customer_date," I again flagged the missing values via column creation (`order_delivered_carrier_date_null_flag` and `order_delivered_customer_date_null_flag`) and next treated those very values based on their respective orders' status, in the following way: for rows where `order_status` was 'delivered', I substituted missing values with 'Info not available' and for all remaining ones I instead replaced missing values with 'Order not delivered', using the assumption that a delivered order's missing value pertaining to either dates of delivery to carrier or final customer was due to mere error in registering said dates whereas a non-delivered order's missing value in such dates would be obviously caused by either an yet incomplete or botched order.

- For all remaining columns, as I do not see how any better and reasonable value that could serve as substitute for the missing cells, the null values will be mantained hence no action will ensure on this respect.

## 1.b.i) Treating the missing data of column "order_approved_at"

In [6]:
# Creating a column to flag the original missing values (1 if NaN, 0 otherwise)
df['order_approved_at_null_flag'] = df['order_approved_at'].isnull().astype(int)

# Ensuring that the 'order_purchase_timestamp' and 'order_approved_at' are datetime objects
df['order_purchase_timestamp'] = dd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = dd.to_datetime(df['order_approved_at'])

# Grouping by unique 'order_id' and taking the first value in each group
df_unique_orders = df.groupby('order_id').first().reset_index()

# Ensuring the columns are still in datetime format after grouping
df_unique_orders['order_purchase_timestamp'] = dd.to_datetime(df_unique_orders['order_purchase_timestamp'])
df_unique_orders['order_approved_at'] = dd.to_datetime(df_unique_orders['order_approved_at'])

# Calculating the time difference between purchase and approval for unique orders
df_unique_orders['time_difference_between_purchase_and_payment_approval'] = (
    df_unique_orders['order_approved_at'] - df_unique_orders['order_purchase_timestamp']
)

# Dropping NaN values and calculate the mean difference in seconds
mean_diff_unique = df_unique_orders['time_difference_between_purchase_and_payment_approval'].dropna().dt.total_seconds().mean().compute()

# Converting the mean difference from seconds to days
mean_diff_days_unique = mean_diff_unique / (24 * 3600)
print(f"Mean difference for unique orders: {mean_diff_days_unique} days")

Mean difference for unique orders: 0.4341289292466538 days


In [7]:
# Filling the missing values on column 'order_approved_at' using the just-calculated mean of difference between date of purchase and date of payment acceptance

import numpy as np

# Converting mean difference (in days) to a timedelta (in seconds)
mean_diff_timedelta = np.timedelta64(int(mean_diff_days_unique * 86400), 's') # Converting mean_diff_days to seconds

# Filling missing values in 'order_approved_at' by adding the mean difference to 'order_purchase_timestamp'
df['order_approved_at'] = df['order_approved_at'].fillna(df['order_purchase_timestamp'] + mean_diff_timedelta)

# Computing the updated dataframe
df = df.compute()

# Checking for remaining missing values
missing_values = df['order_approved_at'].isnull().sum()
print(f"Remaining missing values: {missing_values}")

# Previewing the new flag column
print(df[['order_approved_at', 'order_approved_at_null_flag']].head())

Remaining missing values: 0
    order_approved_at  order_approved_at_null_flag
0 2017-10-02 11:07:15                            0
1 2017-10-02 11:07:15                            0
2 2017-10-02 11:07:15                            0
3 2018-07-26 03:24:27                            0
4 2018-08-08 08:55:23                            0


## 1.b.ii) Treating the missing data of columns "order_delivered_carrier_date", "order_delivered_customer_date"

In [8]:
# List of columns to process

columns_to_process_1 = ['order_delivered_carrier_date', 'order_delivered_customer_date']

# Function to replace missing values in the columns to process (when 'order_status' is 'delivered', the missing values will be substituted with 'Info not available', otherwise the missing values will be substituted with 'Order not delivered')

def process_column_1(df, col):
    # Flagging the original missing values in a new column (1 if NaN, 0 otherwise)
    df[f'{col}_null_flag'] = df[col].isnull().astype(int)

    # Replacing missing values in column when 'order_status' is 'delivered'
    df.loc[(df[col].isna()) & (df['order_status'] == 'delivered'), col] = 'Info not available'

    # Replacing the remaining missing values of column
    df[col] = df[col].fillna("Order not delivered")

    return df

for col in columns_to_process_1:
    df = process_column_1(df, col)
    missing_values = df[col].isnull().sum()
    print(f"Remaining missing values in '{col}': {missing_values}")

    # Printing the first few rows of the newly created flag column
    print(f"First few rows of '{col}_null_flag':")
    print(df[[col, f'{col}_null_flag']].head())  # Shows both the original and the flag column

Remaining missing values in 'order_delivered_carrier_date': 0
First few rows of 'order_delivered_carrier_date_null_flag':
  order_delivered_carrier_date  order_delivered_carrier_date_null_flag
0          2017-10-04 19:55:00                                       0
1          2017-10-04 19:55:00                                       0
2          2017-10-04 19:55:00                                       0
3          2018-07-26 14:31:00                                       0
4          2018-08-08 13:50:00                                       0
Remaining missing values in 'order_delivered_customer_date': 0
First few rows of 'order_delivered_customer_date_null_flag':
  order_delivered_customer_date  order_delivered_customer_date_null_flag
0           2017-10-10 21:25:13                                        0
1           2017-10-10 21:25:13                                        0
2           2017-10-10 21:25:13                                        0
3           2018-08-07 15:27:45     

## <ins>1.c) Elimination of duplicate rows</ins>

We check for duplicate rows in our dataset and eliminate any potentially existing ones as no duplicate row would ever add any value to our analysis, it'd instead complexify it by potentially provoking necessary extra effort as to not skewe calculations by considering the very same information multiple times.

In [9]:
# Dropping duplicate rows and keeping the original DataFrame name
original_row_count = len(df)

# Checking for any duplicate rows and removing them
df.drop_duplicates()

# Calculating how many rows were dropped
cleaned_row_count = len(df)
rows_dropped = original_row_count - cleaned_row_count

print(f"Number of duplicate rows dropped: {rows_dropped}")

Number of duplicate rows dropped: 0


## <ins>1.d) Transformation of categorical variables into numerical ones</ins>

We do so because having categorical variables turned into numerical ones enables analyses of those that would otherwise be impossible. Examples of such would be correlation analysis or predictive modeling.

Before we carry out that transformation we should understand if, for each column containing categorical variables, there is a ranking between those variables or not, as that will influence how the turn to numerical will take place:
- when such a ranking exists, no new columns will be created and instead the categorical variables will be turned into numbers whose dimension signify a hierarchy between the values
- when a ranking doesn't exist, new columns will be created, one per categorical variable, and the original column with the categorical variables will be deleted

Moreover, for categorical columns that appear to have a large number of unique values, it would be sound to understand the sheer scale of those unique values to therefore know if it'd make sense to clusterize those into smaller, hence manageable, groups, before the turn to numerical happens.

Note: Dataframe columns that didn't originally include categorical variables but were given such via our own treatment of their missing values will not have those very strings be turned numerical as those represent missing or incomplete information rather than distinct categories with potential predictive power. Converting those into numerical values could introduce noise into the analysis, as they do not signify a real or measurable feature of the data. Instead, we rely on the flag columns that were previously created to track these cases, ensuring that the distinction between real data and missing information is maintained without distorting the analysis.

In [10]:
# Checking, for each categorical column, if there is hierarchy between the values to then decide if we transform those very columns' categorical variables into numerical variables displayed in one column (happenning when a ranking exists between the variables) or several columns (happenning when a ranking does not exist between the variables).

# We're also checking if there is need to clusterize the categorical variables into smaller 

# List of column names
columns = [
    'order_status',
    'customer_city',
    'customer_state',
    'payment_type',
    'review_comment_title',
    'review_comment_message',
    'product_category_name',
    'seller_city',
    'seller_state',
    'geolocation_city',
    'geolocation_state',
    'product_category_name_english',
    'origin',
    'business_segment',
    'lead_type',
    'lead_behaviour_profile',
    'has_company',
    'has_gtin',
    'average_stock',
    'business_type'
]

# Loop over the columns and print unique values and their count for each
for column in columns:
    unique_values = df[column].unique()
    unique_count = len(unique_values)
    print(f"Unique values for {column}:")
    print(unique_values)
    print(f"Number of unique values in {column}: {unique_count}")
    print()  # Empty line for readability

Unique values for order_status:
<ArrowStringArray>
[  'delivered',    'invoiced',     'shipped',  'processing', 'unavailable',
    'canceled',     'created',    'approved']
Length: 8, dtype: string
Number of unique values in order_status: 8

Unique values for customer_city:
<ArrowStringArray>
[                        <NA>,                 'vianopolis',
                 'ouro preto',                    'goiania',
           'feira de santana',                  'sao paulo',
             'cruz das almas',                  'paranavai',
                      'tocos',                  'capelinha',
 ...
 'santa rita do passa quatro',        'sao joao do oriente',
                   'ouroeste',                   'jesuania',
         'agua fria de goias',                  'sooretama',
              'campo formoso',          'casimiro de abreu',
                     'taruma',                    'sarandi']
Length: 972, dtype: string
Number of unique values in customer_city: 972

Unique values for

## Classification and Clusterization of Hierarchical and High-Cardinality Columns for Data Analysis

From the code above, we've deciphered which categorical columns' values have a hierarchy amongst themselves and which don't. Moreover, we've understood which columns have so many unique values that only a clusterization of these would reasonably allow for further analyses that'd include those very columns. 

Categorical columns whose values have a hierarchy amongst themselves:

- review_comment_title
- review_comment_message
- has_company
- has_gtin

Categorical columns whose values do not have a hierarchy amongst themselves:

- order_status
- customer_city
- customer_state
- payment_type
- product_category_name
- seller_city
- seller_state
- geolocation_city
- geolocation_state
- product_category_name_english
- origin
- business_segment
- lead_type
- lead_behaviour_profile
- average_stock
- business_type

Categorical columns whose number of unique values is big enough that only a clusterization of those would reasonably allow for further analyses:

- customer_city --> this column's clusters are the values in column customer_state, therefore customer_city will be dropped and we'll keep customer_state as its manageable form
- review_comment_title --> this column will be dropped, as the sentiment behind customers’ reviews will already have been given by the more comprehensive hence more accurate review_comment_message
- review_comment_message --> this column will be clustered using sentiment analysis
- seller_city --> this column's clusters are the values in column seller_state, therefore customer_city will be dropped and we'll keep customer_state as its manageable form
- geolocation_city--> this column's clusters are the values in column geolocation_state, therefore customer_city will be dropped and we'll keep customer_state as its manageable form

## Treating the categorical columns whose variables can be ranked between themselves as depicted by the text above

We'll start this section of our analysis by dropping the `review_comment_title` column. Afterwards, we'll start our work on the `review_comment_message` column, which will be the subject of sentiment analysis that'll decode the review-messages and transform them into either 'Positive', 'Negative' or 'No Review'. 

In [11]:
# Dropping the review_comment_title column

df = df.drop(['review_comment_title'], axis=1)

In [12]:
import re
import pandas as pd
from transformers import pipeline, DistilBertTokenizer

# 1. Text Cleaning
def clean_text(text):
    if pd.isna(text) or text.strip() == "":  # Handle missing or empty reviews
        return "no review"  # Label missing reviews
    text = re.sub('<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text.lower()  # Convert to lowercase

# Apply text cleaning to the 'review_comment_message' column
df['cleaned_review'] = df['review_comment_message'].apply(clean_text)

# 2. Calculating review length (after cleaning) to inform tokenization max length
# Calculating the length of each review
df['review_length'] = df['cleaned_review'].apply(lambda x: len(x.split()))

# Getting basic statistics
print(df['review_length'].describe())

# Automatically set max_seq_length to the 95th percentile review length
max_length_95th = df['review_length'].quantile(0.95)
print(f"95th percentile of review lengths: {max_length_95th}")

# Set max_seq_length dynamically based on the 95th percentile
max_seq_length = int(max_length_95th)  # Convert to an integer for tokenization
print(f"Max sequence length for tokenization: {max_seq_length}")

# 3. Load the pre-trained sentiment analysis pipeline (DistilBERT)
sentiment_classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# 4. Performing sentiment analysis in batches
batch_size = 10000  # Adjust based on memory capacity

def process_batch(batch_df):
    # Perform sentiment analysis on the cleaned reviews
    batch_results = sentiment_classifier(batch_df['cleaned_review'].tolist(), truncation=True)
    
    # Store the results in the DataFrame
    batch_df['sentiment_analysis'] = [result['label'] for result in batch_results]
    
    return batch_df

# Process the DataFrame in batches
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch_df = df[start:end].copy()
    
    # Process each batch
    batch_df = process_batch(batch_df)
    
    # Update the main DataFrame with the processed batch
    df.loc[start:end - 1, 'sentiment_analysis'] = batch_df['sentiment_analysis']
    
    print(f"Processed batch from {start} to {end}")

# Display the cleaned and sentiment-analyzed DataFrame
print(df.head(20))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


count    6.080924e+06
mean     1.197814e+01
std      1.092003e+01
min      0.000000e+00
25%      2.000000e+00
50%      8.000000e+00
75%      2.100000e+01
max      4.200000e+01
Name: review_length, dtype: float64
95th percentile of review lengths: 34.0
Max sequence length for tokenization: 34
Processed batch from 0 to 10000
Processed batch from 10000 to 20000
Processed batch from 20000 to 30000
Processed batch from 30000 to 40000
Processed batch from 40000 to 50000
Processed batch from 50000 to 60000
Processed batch from 60000 to 70000
Processed batch from 70000 to 80000
Processed batch from 80000 to 90000
Processed batch from 90000 to 100000
Processed batch from 100000 to 110000
Processed batch from 110000 to 120000
Processed batch from 120000 to 130000
Processed batch from 130000 to 140000
Processed batch from 140000 to 150000
Processed batch from 150000 to 160000
Processed batch from 160000 to 170000
Processed batch from 170000 to 180000
Processed batch from 180000 to 190000
Processe

In [13]:
# We now turn all the 'POSITIVE' and 'NEGATIVE' comments on the newly-formed sentiment_analysis column into 1s and 0s. 

# Map 'POSITIVE' to 1 and 'NEGATIVE' to 0
df['sentiment_analysis'] = df['sentiment_analysis'].map({'POSITIVE': 1, 'NEGATIVE': 0})

# Convert the column to float, if needed
df['sentiment_analysis'] = df['sentiment_analysis'].astype(float)

df.head(20)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,product_category_name_english,mql_id,first_contact_date,landing_page_id,origin,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,order_approved_at_null_flag,order_delivered_carrier_date_null_flag,order_delivered_customer_date_null_flag,cleaned_review,review_length,sentiment_analysis
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,31570.0,-19.807681,-43.980427,belo horizonte,MG,perfumery,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,0,0,0,muito bom o produto,4,0.0
4,47770eb9100c2d0c4494

In [15]:
# And now we turn all the 'true' and 'false' values of 'has_company' and 'has_gtin' into 1s and 0s:

binary_columns = ['has_company', 'has_gtin']

for column in binary_columns:
    # Convert 'True'/'False' strings to boolean type first, then to integer (1/0)
    df[column] = df[column].astype(bool).astype(int)

df.head(20)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,product_category_name_english,mql_id,first_contact_date,landing_page_id,origin,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,order_approved_at_null_flag,order_delivered_carrier_date_null_flag,order_delivered_customer_date_null_flag,cleaned_review,review_length,sentiment_analysis
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,9350.0,-23.680729,-46.444238,maua,SP,housewares,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,NaN,NaN,0,0,0,no testei o produto ainda mas ele veio correto...,32,0.0
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,<NA>,NaN,<NA>,<NA>,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,31570.0,-19.807681,-43.980427,belo horizonte,MG,perfumery,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,NaN,NaN,0,0,0,muito bom o produto,4,0.0
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b

In [16]:
# We now drop the categorical columns that are too granular in their values to be manageable and that simultaneously already have corresponding columns that cluster those granular values:

df = df.drop(['customer_city', 'seller_city', 'geolocation_city'], axis=1)

In [22]:
# And finally we turn the categorical columns whose values do not have a hierarchy amongst themselves into new binary columns using one-hot encoding:

columns_to_encode = [
    'order_status', 
    'customer_state', 
    'payment_type', 
    'product_category_name', 
    'seller_state', 
    'geolocation_state', 
    'product_category_name_english', 
    'origin', 
    'business_segment', 
    'lead_type', 
    'lead_behaviour_profile', 
    'business_type'
]

# Perform one-hot encoding and add the resulting columns to the original DataFrame
df = pd.get_dummies(df, columns=columns_to_encode, drop_first=False)

# Define a single prefix to apply to all one-hot encoded columns
prefix = "encoded"

# Rename the columns to include the prefix
df = df.rename(columns=lambda x: f"{prefix}_{x}" if any(col in x for col in columns_to_encode) else x)

# Display the first 5 rows of the updated DataFrame
df.head(5)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_installments,payment_value,review_id,review_score,review_comment_message,review_creation_date,review_answer_timestamp,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,mql_id,first_contact_date,landing_page_id,sdr_id,sr_id,won_date,has_company,has_gtin,average_stock,declared_product_catalog_size,declared_monthly_revenue,order_approved_at_null_flag,order_delivered_carrier_date_null_flag,order_delivered_customer_date_null_flag,cleaned_review,review_length,sentiment_analysis,order_status_approved,order_status_canceled,order_status_created,order_status_delivered,order_status_invoiced,order_status_processing,order_status_shipped,order_status_unavailable,customer_state_AC,customer_state_AL,customer_state_AM,customer_state_AP,customer_state_BA,customer_state_CE,customer_state_DF,customer_state_ES,customer_state_GO,customer_state_MA,customer_state_MG,customer_state_MS,customer_state_MT,customer_state_PA,customer_state_PB,customer_state_PE,customer_state_PI,customer_state_PR,customer_state_RJ,customer_state_RN,customer_state_RO,customer_state_RR,customer_state_RS,customer_state_SC,customer_state_SE,customer_state_SP,customer_state_TO,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_not_defined,payment_type_voucher,product_category_name_agro_industria_e_comercio,product_category_name_alimentos,product_category_name_alimentos_bebidas,product_category_name_artes,product_category_name_artes_e_artesanato,product_category_name_artigos_de_festas,product_category_name_artigos_de_natal,product_category_name_audio,product_category_name_automotivo,product_category_name_bebes,product_category_name_bebidas,product_category_name_beleza_saude,product_category_name_brinquedos,product_category_name_cama_mesa_banho,product_category_name_casa_conforto,product_category_name_casa_conforto_2,product_category_name_casa_construcao,product_category_name_cds_dvds_musicais,product_category_name_cine_foto,product_category_name_climatizacao,product_category_name_consoles_games,product_category_name_construcao_ferramentas_construcao,product_category_name_construcao_ferramentas_ferramentas,product_category_name_construcao_ferramentas_iluminacao,product_category_name_construcao_ferramentas_jardim,product_category_name_construcao_ferramentas_seguranca,product_category_name_cool_stuff,product_category_name_dvds_blu_ray,product_category_name_eletrodomesticos,product_category_name_eletrodomesticos_2,product_category_name_eletronicos,product_category_name_eletroportateis,product_category_name_esporte_lazer,product_category_name_fashion_bolsas_e_acessorios,product_category_name_fashion_calcados,product_category_name_fashion_esporte,product_category_name_fashion_roupa_feminina,product_category_name_fashion_roupa_infanto_juvenil,product_category_name_fashion_roupa_masculina,product_category_name_fashion_underwear_e_moda_praia,product_category_name_ferramentas_jardim,product_category_name_flores,product_category_name_fraldas_higiene,product_category_name_industria_comercio_e_negocios,product_category_name_informatica_acessorios,product_category_name_instrumentos_musicais,product_category_name_la_cuisine,product_category_name_livros_importados,product_category_name_livros_interesse_geral,product_category_name_livros_tecnicos,product_category_name_malas_acessorios,product_category_name_market_place,product_category_name_moveis_colchao_e_estofado,product_category_name_moveis_cozinha_area_de_servico_jantar_e_jardim,product_category_name_moveis_decoracao,product_category_name_moveis_escri

In [3]:
# Select columns with the specified prefix
binary_columns = [col for col in df.columns if col.startswith(prefix)]

for column in binary_columns:
    df[column] = df[column].astype(int)

df.head()

NameError: name 'df' is not defined

## <ins>1.e) Feature engineering</ins>

This section constructs new features to enrich the dataset and provide deeper insights for analysis. We begin by calculating time intervals within the order delivery process, flagging orders as 'early,' 'on-time,' or 'late' and extracting dates and times from each order. Next, we analyze customer purchase behavior, summarizing frequency, recency, spending patterns, and identifying top purchasers. Product and seller popularity metrics are derived, including order counts and average review scores. We also assess average distances between customers and sellers, with geographical features to capture state-level order trends.

Additionally, we engineer lead and marketing features, quantify customer review lengths to gauge engagement, and calculate product category popularity. Payment-related features cover consistency between payment values and prices, as well as installment preferences. Finally, we drop the original categorical columns used in feature creation and calculate a total null count per row as a data quality indicator. Together, these features create a comprehensive view of customer behaviors, product demand, logistics, and payment patterns.

In [ ]:
# 1. Calculating the time-intervals between the various check-points of the order-delivery process, flagging all orders as 'early', 'late' or 'on-time' and identifying which year, month of year, day of week and time of day each order pertains to

# Ensure datetime columns are in datetime format
datetime_cols = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date', 
    'first_contact_date', 
    'won_date'
]

for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Differences between key dates
df['approved_to_purchase_duration'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # in hours
df['carrier_to_approved_duration'] = (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / 3600
df['customer_to_carrier_duration'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 3600
df['customer_to_purchase_duration'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600

# Flag orders based on estimated vs actual delivery date
df['delivery_status'] = df.apply(
    lambda row: 'late' if row['order_delivered_customer_date'] > row['order_estimated_delivery_date'] else
    ('early' if row['order_delivered_customer_date'] < row['order_estimated_delivery_date'] else 'on_time'),
    axis=1
)

# Year, month, day of week, and time of day for purchase and delivery dates
df['purchase_year'] = df['order_purchase_timestamp'].dt.year
df['purchase_month'] = df['order_purchase_timestamp'].dt.month
df['purchase_day_of_week'] = df['order_purchase_timestamp'].dt.dayofweek  # 0 = Monday, 6 = Sunday
df['purchase_time_of_day'] = df['order_purchase_timestamp'].dt.hour

df['delivery_year'] = df['order_delivered_customer_date'].dt.year
df['delivery_month'] = df['order_delivered_customer_date'].dt.month
df['delivery_day_of_week'] = df['order_delivered_customer_date'].dt.dayofweek
df['delivery_time_of_day'] = df['order_delivered_customer_date'].dt.hour

# Preview the new dataframe with all features
df.head()

In [ ]:
# 2. Calculating, by customer, their orders' purchase-frequency, recency, overall purchase value, average purchase value, average order-purchase interval (in days) & attributing labels to the top 25% of purchasers in frequency and value or orders

from datetime import datetime

# Set a reference date for calculating recency (e.g., today's date)
reference_date = datetime.now()

# Group by customer_id to calculate frequency, recency, and monetary value
customer_segmentation = df.groupby('customer_id').agg(
    frequency=('order_id', 'nunique'),  # Unique number of orders per customer
    recency=('order_purchase_timestamp', lambda x: (reference_date - x.max()).days),  # Days since last purchase
    monetary=('payment_value', 'sum')   # Total amount spent by each customer
).reset_index()

# Preview the customer segmentation data
customer_segmentation.head()

# Calculate average purchase value and average order interval
customer_segmentation['avg_purchase_value'] = customer_segmentation['monetary'] / customer_segmentation['frequency']

# Sort the DataFrame
df = df.sort_values(by=['customer_id', 'order_purchase_timestamp'])

# Drop duplicates based on the order identifier
df_unique = df.drop_duplicates(subset=['customer_id', 'order_id'])

# Calculate the time difference
df_unique['days_between_orders'] = df_unique.groupby('customer_id')['order_purchase_timestamp'].diff().dt.days

# Calculate the average order interval by customer
avg_order_interval = df_unique.groupby('customer_id')['days_between_orders'].mean().reset_index(name='avg_order_interval')

# Merge this back with the customer segmentation data
customer_segmentation = customer_segmentation.merge(avg_order_interval, on='customer_id', how='left')

# Fill NaN values in avg_order_interval with 0 for customers with only one purchase
customer_segmentation['avg_order_interval'] = customer_segmentation['avg_order_interval'].fillna(0)

# Preview the enhanced segmentation data
customer_segmentation.head()

# Define thresholds for customer segmentation
high_value_threshold = customer_segmentation['monetary'].quantile(0.75)  # Top 25% spenders
high_frequency_threshold = customer_segmentation['frequency'].quantile(0.75)  # Top 25% frequent customers

# Create labels
def assign_segment(row):
    if row['monetary'] >= high_value_threshold and row['frequency'] >= high_frequency_threshold:
        return 'Loyal High-Spender'
    elif row['monetary'] >= high_value_threshold:
        return 'High-Spender'
    elif row['frequency'] >= high_frequency_threshold:
        return 'Frequent Buyer'
    else:
        return 'Occasional Buyer'

customer_segmentation['segment'] = customer_segmentation.apply(assign_segment, axis=1)

# Preview the segmentation with labels
customer_segmentation.head()

# Merge all columns of customer_segmentation_df into the main df
df = df.merge(customer_segmentation_df, on='customer_id', how='left')

# Preview to ensure the merge was successful
df.head()

In [ ]:
# 3. Calculating the number of times each product was ordered (product popularity) and its average review score, the number of times each seller was ordered a product (seller popularity) and average review score for each of those sellers and finally the average delivery duration of each product and seller 

# Calculate product popularity (number of times each product is ordered)
product_popularity = df.groupby('product_id').size().reset_index(name='product_popularity')

# Calculate average review score for each product
product_avg_review_score = df.groupby('product_id')['review_score'].mean().reset_index(name='product_avg_review_score')

# Merge these new features into the main DataFrame
df = df.merge(product_popularity, on='product_id', how='left')
df = df.merge(product_avg_review_score, on='product_id', how='left')

# Calculate seller popularity (number of orders handled by each seller)
seller_popularity = df.groupby('seller_id').size().reset_index(name='seller_popularity')

# Calculate average review score for each seller
seller_avg_review_score = df.groupby('seller_id')['review_score'].mean().reset_index(name='seller_avg_review_score')

# Merge these features into the main DataFrame
df = df.merge(seller_popularity, on='seller_id', how='left')
df = df.merge(seller_avg_review_score, on='seller_id', how='left')

# Calculate delivery duration (in days) for each order
df['delivery_duration_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

# Calculate average delivery duration for each product
product_avg_delivery_duration = df.groupby('product_id')['delivery_duration_days'].mean().reset_index(name='product_avg_delivery_duration')

# Merge the average delivery duration feature into the main DataFrame
df = df.merge(product_avg_delivery_duration, on='product_id', how='left')

# Calculate average delivery duration for each seller
seller_avg_delivery_duration = df.groupby('seller_id')['delivery_duration_days'].mean().reset_index(name='seller_avg_delivery_duration')

# Merge the average delivery duration feature for sellers into the main DataFrame
df = df.merge(seller_avg_delivery_duration, on='seller_id', how='left')

# Preview the DataFrame with all new Product and Seller features
df.head()

In [ ]:
# 4. Calculating the average distance between customer and seller then using that calculation to calculate the average order distance & total count of orders by both customer_state and seller_state

from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance based on lat/lng using the Haversine formula
def haversine_distance(lat1, lng1, lat2, lng2):
    # Radius of Earth in kilometers
    R = 6371.0
    
    # Convert degrees to radians
    lat1, lng1, lat2, lng2 = map(radians, [lat1, lng1, lat2, lng2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlng / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Apply Haversine formula to each row to calculate customer-seller distance
df['distance_km'] = df.apply(
    lambda row: haversine_distance(row['customer_lat'], row['customer_lng'], row['seller_lat'], row['seller_lng']), axis=1
)

# Regional Flag: Check if customer and seller are in the same state
df['same_state'] = (df['customer_state'] == df['seller_state']).astype(int)

# Aggregate Features by State

# Calculate the average distance per unique order, then get the mean by customer state
avg_distance_by_order = df.groupby(['customer_state', 'order_id'])['distance_km'].mean()
avg_distance_by_state = avg_distance_by_order.groupby('customer_state').mean().rename('avg_distance_by_state')
df = df.merge(avg_distance_by_state, on='customer_state', how='left')

# Count unique orders by customer state
order_count_by_state = df.groupby('customer_state')['order_id'].nunique().rename('order_count_by_state')
df = df.merge(order_count_by_state, on='customer_state', how='left')

# Preview the new geolocation-based features
df[['distance_km', 'same_state', 'avg_distance_by_state', 'order_count_by_state']].head()

In [ ]:
# 5. Calculating Lead and Marketing Features

# Count unique lead types for each customer
leads_per_customer_df = df.groupby('customer_id')['lead_type'].nunique().rename('unique_leads_customer_count')

# Merge the result back to the original DataFrame
df = df.merge(leads_per_customer_df, on='customer_id', how='left')

# Count unique lead types for each order
leads_per_order_df = df.groupby('order_id')['lead_type'].nunique().rename('unique_leads_order_count')

# Merge the result back to the original DataFrame
df = df.merge(leads_per_order_df, on='order_id', how='left')

# Conversion rate of Marketing Qualified Leads

# Count unique leads per 'mql_id' 
unique_leads_by_mql = df.groupby('mql_id')['lead_type'].nunique().rename('unique_leads_count')

# Count unique orders per 'mql_id' 
unique_orders_by_mql = df.groupby('mql_id')['order_id'].nunique().rename('unique_orders_count')

# Merge the counts to calculate conversion rates
mql_conversion_df = unique_orders_by_mql.to_frame().join(unique_leads_by_mql)

# Calculate the conversion rate (order count / lead count)
mql_conversion_df['conversion_rate'] = mql_conversion_df['unique_orders_count'] / mql_conversion_df['unique_leads_count']

# Preview the conversion rates
mql_conversion_df[['unique_leads_count', 'unique_orders_count', 'conversion_rate']].head()

In [ ]:
# 6. Calculating the lengths of customer reviews to determine customer engagement

# Calculate the length of each review in characters
df['review_length'] = df['review_comment_message'].apply(lambda x: len(x) if isinstance(x, str) else 0)

# Define engagement categories based on review length
def categorize_engagement(length):
    if length < 50:
        return 'Low'
    elif 50 <= length <= 150:
        return 'Medium'
    else:
        return 'High'

# Apply engagement categorization
df['engagement_level'] = df['review_length'].apply(categorize_engagement)

# Convert engagement categories to numerical labels: Low -> 0, Medium -> 1, High -> 2
engagement_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df['engagement_level_numeric'] = df['engagement_level'].map(engagement_mapping)

# Preview the DataFrame with the new engagement features
df[['review_comment_message', 'review_length', 'engagement_level', 'engagement_level_numeric']].head()

In [ ]:
# 7. Calculating the popularity of every product category

# Count the unique number of order_id values per product category (category popularity)
category_popularity = df.groupby('product_category_name')['order_id'].nunique().rename('category_popularity')

# Merge the popularity metric back to the main DataFrame
df = df.merge(category_popularity, on='product_category_name', how='left')

# Preview the new popularity feature
df[['product_category_name', 'category_popularity']].head()

In [ ]:
# 8. Calculating Payment-based features: the payment consistency (meaning difference between an order's total payment value and its price) and flagging both single and multiple-installments orders

# Sum payment_value for each order to compare with total price
order_payment_sum = df.groupby('order_id')['payment_value'].sum()

# Merge this summed value back to the original DataFrame to enable comparison
df = df.merge(order_payment_sum.rename('total_payment_value'), on='order_id', how='left')

# Calculate Payment Consistency at the order level
df['payment_consistency'] = (df['total_payment_value'] == df['price']).astype(int)

# Create Installment-Based Indicators at the order level
# First, flag orders with any installments greater than 1
installment_flag = df.groupby('order_id')['payment_installments'].max() > 1
df = df.merge(installment_flag.rename('uses_installments').astype(int), on='order_id', how='left')

# Flag single payment (when no installments are used)
df['single_payment'] = (df['uses_installments'] == 0).astype(int)

df.head()

In [ ]:
# 9. Dropping the still-existing categorical columns

df = df.drop(columns=['review_comment_message', 'cleaned_review'])

In [ ]:
# 10. Flagging the total number of null values on each column

# Count total null values for each row and add as a new feature
df['null_count'] = df.isnull().sum(axis=1)

# Display the first 5 rows to verify the new column
df.head()